<a href="https://colab.research.google.com/github/hannape/IBAC-Biodiv/blob/master/01_Train_Matrix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Stworzenie macierzy treningowej, zbalansowanej. Dla nagrań z 2016,17,18 powinna mieć pierwszy wymiar 15904

Kod wzięty z I2: Copy of 001-SplitRecordingAsFunction_3repr_IBAC.ipynb, wyczyszczony z niepotrzebnych rzeczy


In [0]:
## JAKO FUNKCJA

!pip install audiosegment
!pip install librosa
!pip install pydub
!pip install webrtcvad
!pip install librosa
!pip install -q spectrum
!pip install pkgconfig libtfr

In [0]:
import libtfr
import librosa
from scipy.io import wavfile
from random import choice
import librosa
from pydub import AudioSegment
from pydub.utils import make_chunks
import math
import audiosegment
from pydub import AudioSegment
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import preprocessing
import glob
import os
import scipy
import librosa
from librosa import display
from librosa.core import power_to_db
from librosa.core import amplitude_to_db
import IPython.display as ipd

In [0]:
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks')

In [0]:
#### Parametry ####
             
plot_repr = 0          # czy wyrysowywać głosy wszystkie
play_sound = 0
minimal_duration = 4   # minimalny czas trwania głosu w ms, b zakwalifikowac chunka jako hasbird
f_min = 4000           # minimalna częstotliwość analizowana ze spektro
f_max = 9500           # maksymalna częstotliwość analizowana ze spektro
f_min2 = 4000           # minimalna częstotliwość analizowana ze spektro
f_max2 = 9500           # maksymalna częstotliwość analizowana ze spektro
n_mels = 60
n_mels2 = 80
step = 150

###################

def representations(frame):
  
      ####### 
      global result1,result1b,result3,result3b,result5,result5b
      
      fs = 44100
      samples = frame.get_array_of_samples()
      samples_float = librosa.util.buf_to_float(samples,n_bytes=2,dtype=np.float32)
      audio = np.array(samples_float)
      '''
      ####### reprezentacja 1  ------- 63 x 148 ------
      stft = librosa.stft(audio, n_fft=512, win_length=512, hop_length=150, window= "hann") #scipy.signal.get_window(('gaussian',50) ,512) )#scipy.signal.gaussian(512, 2)   ) 
      stft1 = amplitude_to_db(np.abs(stft)**2)
      freqs = librosa.core.fft_frequencies(n_fft=512, sr=44100)
      fmin =  min(freqs[(freqs >= f_min)])
      i,  = np.where( freqs >= min(freqs[(freqs >= f_min)]))
      j,  = np.where( freqs <= max(freqs[(freqs <= f_max)]))
      stft1 = stft1[min(i):max(j),]
      result1 = stft1
      
      ####### reprezentacja 1b - spektrogram większy hop length ------- 63 x 63 ------
      stft = librosa.stft(audio, n_fft=512, win_length=512, hop_length=355, window= "hann") #scipy.signal.get_window(('gaussian',50) ,512) )#scipy.signal.gaussian(512, 2)   ) 
      stft1 = amplitude_to_db(np.abs(stft)**2)
      freqs = librosa.core.fft_frequencies(n_fft=512, sr=44100)
      fmin =  min(freqs[(freqs >= f_min)])
      i,  = np.where( freqs >= min(freqs[(freqs >= f_min)]))
      j,  = np.where( freqs <= max(freqs[(freqs <= f_max)]))
      stft1 = stft1[min(i):max(j),]
      result1b = stft1
      
      ####### reprezentacja 3 - mel spektrogram ------- 60 x 111 ------
    
      # Compute Short-Term Fourier Transform (STFT).
      stft = librosa.stft(np.array(samples_float), n_fft=512, win_length=512, hop_length= 200,    window="hann")

      # Compute squared magnitude coefficients.
      abs2_stft = (stft.real*stft.real) + (stft.imag*stft.imag)
      result3 = librosa.feature.melspectrogram(y=None, S=abs2_stft, sr=44100, n_mels= 60, fmin = 4000, fmax=9500, hop_length=512, n_fft=512)
      result3 = 0.5 * librosa.amplitude_to_db(result3, ref=1.0)
      
      ####### reprezentacja 3b - mel spektrogram większy hop length ------- 60 x 63 ------
    
      # Compute Short-Term Fourier Transform (STFT).
      stft = librosa.stft(np.array(samples_float), n_fft=512, win_length=512, hop_length= 355,    window="hann")

      # Compute squared magnitude coefficients.
      abs2_stft = (stft.real*stft.real) + (stft.imag*stft.imag)
      result3 = librosa.feature.melspectrogram(y=None, S=abs2_stft, sr=44100, n_mels= 60, fmin = 4000, fmax=9500, hop_length=512, n_fft=512)
      result3b = 0.5 * librosa.amplitude_to_db(result3, ref=1.0)
      
      '''
      ####### reprezentacja 3 V3 - mel spektrogram  ------- 60 x 148 ------
    
      # Compute Short-Term Fourier Transform (STFT).
      stft = librosa.stft(np.array(samples_float), n_fft=512, win_length=512, hop_length= 150,    window="hann")

      # Compute squared magnitude coefficients.
      abs2_stft = (stft.real*stft.real) + (stft.imag*stft.imag)
      result3 = librosa.feature.melspectrogram(y=None, S=abs2_stft, sr=44100, n_mels= 60, fmin = 4000, fmax=9500, hop_length=512, n_fft=512)
      result3 = 0.5 * librosa.amplitude_to_db(result3, ref=1.0)
      '''
      ####### reprezentacja 5 - multitaper ------- 64 x 61 ------
      
      result5 = libtfr.tfr_spec(audio, N = 512, step = 355, Np = 490, K = 2, tm = 1, flock =0.1, tlock = 10)     
      freqs, ind = libtfr.fgrid(fs, 512, fpass=(4000,9500)) 
      result5 = result5[ind,]; # tylko interesujące nas pasmo
      
      if(np.isinf(result5).any()):
        print("inf result B: " + str(i))
        
            
      ####### reprezentacja 5b - multitaper o większej rozdzielczości ------- 64 x 149 ------
      
      result5b = libtfr.tfr_spec(audio, N = 512, step = 145, Np = 490, K = 2, tm = 1, flock =0.1, tlock = 10)     
      freqs, ind = libtfr.fgrid(fs, 512, fpass=(4000,9500)) 
      result5b = result5b[ind,]; # tylko interesujące nas pasmo
      
      if(np.isinf(result5b).any()):
        print("inf result B: " + str(i))  
      '''

In [0]:
def SplitRecordingFunction(nazwa):

  name_wav = nazwa + ".wav"
  name_txt = nazwa + ".txt" #dot.txt"  txt przecinkowe

  with open((source_labels + name_txt), 'r+') as f:  # zmiana na kropki
      text = f.read()
      f.seek(0)
      f.truncate()
      f.write(text.replace(',', '.'))
  fs, data = wavfile.read(source_audio + name_wav)   # wypluwa błąd, nie rozumie opisow dodanych przez Audacity pewnie
  print(name_wav)
  myaudio = audiosegment.from_file(source_audio + name_wav)
  channel_count = myaudio.channels    #Get channels
  sample_width = myaudio.sample_width #Get sample width
  duration_in_ms = len(myaudio) #Length of audio in ms
  sample_rate = myaudio.frame_rate
  nr_of_chunks =  1 + (duration_in_ms - chunk_length_ms) / (chunk_length_ms - chunk_overlap)

  ########## Odczyt labeli z csv, wyczyszczenie zaznaczonych trzasków (t, ?)
  
  calls_to_cut = ['t', 't?', 'g', 'czapla', 'gh', 'g cz', 'puszczyk','gaski','g?','mewa?','zwierzak?','high freq','g niskie','??? mysz']
  
  ###### tu nie wycięliśmy 't ' i 't  ', a w teście tak, mam nadzieję że to jest marginalny problem, niech już tak zostanie.
  
  if (os.stat(source_labels + name_txt).st_size != 0):
    y_read = pd.read_csv(source_labels + name_txt,  header=None, sep = "\t")
    y_read.columns = ["start", "end", "bird"]
    y_read_cleaned = y_read[~y_read['bird'].isin(calls_to_cut)]
    print(y_read_cleaned['bird'])
    y_read = y_read_cleaned
    not_empty = 1
  else:
    print(source_labels + name_txt)
    print("bez etykiet!!! : " + str(name_wav))
    not_empty = 0
    sum_birds2 = 0
    birds_chunks = []
    df=pd.DataFrame()
  
  ########## Wyszukanie pozytywnych fragmentów - fragmentów zawierający głos, ilość takich ramek

  image_positive = []
  description_positive = []
  species=[]
  sum_birds=0
  has_bird = 1
  plot_repr = 0          # czy wyrysowywać głosy wszystkie
  play_sound = 1
  
  if (not_empty):
    for i in range(0,math.floor(nr_of_chunks)): #(2300,2600): #math.floor(nr_of_chunks)): 
      frame_start = i*(chunk_length_ms - chunk_overlap)
      frame_end = i*(chunk_length_ms - chunk_overlap)+ chunk_length_ms
      frame = myaudio[frame_start : frame_end]
      x = range(frame_start,frame_end)

      for j in y_read.index.tolist():
        bird_start = int(round(y_read.start[j]*1000))
        bird_end = int(round(y_read.end[j]*1000))
        y = range(bird_start,bird_end)

        intersect_start = max(x[0], y[0])
        intersect_end = min(x[-1], y[-1])+1
        range1 = (range(intersect_start, intersect_end))

        if len(range1)> minimal_duration: 
          sum_birds = sum_birds+1

          #representations(frame)  
          #image_positive.append(result1)  # nie musi być tutaj, dopiero po usunięciu duplikatów może?
          species.append(y_read.bird[j])
          description_positive.append([i, frame_start, frame_end, intersect_start, intersect_end, bird_start, bird_end, has_bird, 0, 0])


  #representations(frame)  # jeden przykładowy, ostatni,  by poznać wymiary reprezentacji       
  
  ########## Usunięcie duplikatów, zakładamy że nie więcej niż 2 głosy w ramce

  labels = ['Chunk_nr', 'Chunk_start', 'Chunk_end', 'Call_start_chunk','Call_end_chunk','Call_start','Call_end','Has_bird','2nd_call_start','2nd_call_end']
  description_pd = pd.DataFrame.from_records(description_positive, columns=labels)   # na lepszy format

  duplic = description_pd[description_pd.duplicated(subset='Chunk_nr',keep=False)]   # wypisanie duplikujących sie ramek (ale z innym glosem zawartym)
  duplic_index = duplic.drop_duplicates(subset='Chunk_nr',keep='first')['Chunk_nr'].index.tolist() 

  #print(duplic)
  #print(duplic_index)

  df = description_pd.drop_duplicates(subset='Chunk_nr',keep='first')  # usuwamy drugie duplikaty z listy
  for i in duplic_index:
    df['2nd_call_start'][i]= description_pd['Call_start'][i+1]   # dopisujemy drugi głos do ramki
    df['2nd_call_end'][i]= description_pd['Call_end'][i+1]
     
  ########## Macierz wyjściowa obrazów i opisów, same ptaki
 
  #description_positive_no_duplic = df   # mamy opisy, czas na reprezentacje:
  rep1 = []
  rep3 = []
  rep5 = []
  spec = []

  for i in df.index: #range(0,np.shape(description_positive_no_duplic)[0]):
      frame = myaudio[df.Chunk_start[i]:df.Chunk_end[i]]
      representations(frame)
      #rep1.append(result1)
      rep3.append(result3)
      #rep5.append(result5)
      spec.append(species[i])
  #print(np.shape(spec))  
  
  ########## Indexy pustych ramek

  '''
  min_chunks = 50  # jeśli głosów w nagraniu jest mniej niż 50, to i tak dobierzemy 50 pustych fragmentów - by choć wziąć próbkę tych warunków
  import random
  random.seed(667)
  sum_birds2 = np.shape(df)[0]  # bez powtórzeń już

  if sum_birds2<min_chunks:
    sum_birds2 = min_chunks

  #print(sum_birds)
  birds_chunks = df['Chunk_nr'].tolist()
  range_nonbirds = [i for i in range(0, math.floor(nr_of_chunks)) if i not in birds_chunks]
  nobirds_chunks = random.sample(range_nonbirds, sum_birds2)  # dobieramy tyle ramek bez ptaków, co mamy z ptakami. Losowo
  #print(nobirds_chunks)
  '''
  
  ### Ładujemy chunki z podziałów, by zawsze te same randomowe negatywy były
  dane1 = np.load(source_npz + 'Kopia '+nazwa + '_7rep.npz' )
  nobirds_chunks = dane1["nobirds_chunks"]
  #print(nobirds_chunks)
  
  ########## Dodanie macierzy z ramkami bez ptaków
  description_negative = []

  for i in nobirds_chunks: 
    frame_start = i*(chunk_length_ms - chunk_overlap)
    frame_end = i*(chunk_length_ms - chunk_overlap)+ chunk_length_ms
    frame = myaudio[frame_start : frame_end]
    representations(frame)
    #rep1.append(result1)
    rep3.append(result3)
    #rep5.append(result5)
    description_negative.append([i, frame_start, frame_end, 0, 0, 0, 0, 0, 0, 0])

  #print(np.shape(rep1))  
  print(np.shape(rep3)) 
  
  ########## Złożenie dwóch macierzy i zapis
  description_negative_pd = pd.DataFrame.from_records(description_negative, columns=labels)

  #final_dataset = np.concatenate([image_positive_no_duplic, np.array(image_negative)])
  final_description = pd.concat([df,description_negative_pd])

  #np.save((source_out + nazwa + 'balanced_dataset' ),final_dataset)
  #np.savez((source_out + nazwa + name_out ), rep1 = rep1, rep3 = rep3, rep5 = rep5,  final_description=final_description, labels=labels, nobirds_chunks=nobirds_chunks,spec = spec) 
  np.savez((source_out + nazwa + name_out ), rep3 = rep3, final_description=final_description, labels=labels, nobirds_chunks=nobirds_chunks,spec = spec) 
 

In [0]:

################################################################ MAIN

source_labels = "drive/My Drive/labele/2018/"# 2016_50/"
source_npz = "drive/My Drive/repr7_2018/"
source_out = "drive/My Drive/"
source_audio = "drive/My Drive/nagrania/2018/"

nazwa_list=[]
name_out = '_1repV3'

a = sorted(os.listdir(source_labels))      ## nie było wcześniej sorted, więc w różnej kolejności były wyliczane
for i in range(0,np.shape(a)[0]):
  nazwa_list.append(os.path.splitext(a[i])[0])
  
print(nazwa_list)

#### Parametry ####
chunk_length_ms = 500
chunk_overlap = 150
###################  


print(np.shape(nazwa_list)[0])

In [0]:
print(np.shape(nazwa_list)[0])
for i in range(0,np.shape(nazwa_list)[0]):
  
  print("nagranie nr "+ str(i) + ": " + str(nazwa_list[i]))
  SplitRecordingFunction(nazwa_list[i])

In [0]:
source_repr = 'drive/My Drive/'
folder_main = [ 'repr1V3_2016/','repr1V3_2017/','repr1V3_2018/' ]   ## foldery gdzie są pliki npz z poprzedniego etapu. tyle ile nagrań, każdy zawiera X reprezentacji

X_train_all = []
y_train_all = []
y_train_species_all = []
repr_to_check = ['rep3']

#for k in range(0, np.size(repr_to_check)):
for k in range(0, np.size(folder_main)):
  
  
  repr_nr = repr_to_check[0]
  print('-----------' + repr_nr + '-----------')
  print('-----------' + folder_main[k] + '-----------')
  
  # pierwsze dane, by mieć kształt macierzy i kolejno wcztywać następne
  records = sorted(os.listdir(source_repr + folder_main[k]))
  print(np.shape(records))
  
  dane1 = np.load(source_repr + folder_main[k] + records[0], allow_pickle=True )
  X_train= dane1[repr_nr]
  y_train = dane1['final_description'][:,7]
  y_train_species = dane1['spec'] 
  print(records[0])
  print(np.shape(X_train))
  print(np.shape(y_train))
  print(np.shape(y_train_species))
  print('Dostepne dane: ' + str(dane1.files))
  
  for i in range (1,np.size(records)):
    print(records[i])
    dane1 = np.load(source_repr + folder_main[k] + records[i], allow_pickle=True )
    X_train = np.concatenate((X_train,dane1[repr_nr]))
    y_train = np.concatenate((y_train,dane1['final_description'][:,7]))
    #print(np.shape(y_test_species))
    
    print(np.shape(X_train))
    print(np.shape(y_train))
    a = dane1['spec']
    #print(len(a.shape))
    if (len(a.shape)==2):  # jeśli dwa wymiary
       a = np.squeeze(dane1['spec'])  # to squeezujemy
  
    y_train_species = np.concatenate((y_train_species,a )) 
    print(np.shape(y_train_species))
  
  
  print(np.shape(X_train))
  print(np.shape(y_train))
  print(np.shape(y_train_species))
  if k==0:
      X_train_all = X_train
      y_train_all = y_train
      y_train_species_all = y_train_species
  else: 
      X_train_all = np.concatenate((X_train_all,X_train))
      y_train_all = np.concatenate((y_train_all,y_train))
      y_train_species_all = np.concatenate((y_train_all,y_train_species))

np.save(('drive/My Drive/X_train_' + repr_nr+ 'V3'), X_train_all)
  
np.save(('drive/My Drive/y_train_' + repr_nr + 'V3'), y_train_all)
np.save(('drive/My Drive/y_train_species_'+ repr_nr+ 'V3'), y_train_species_all)

print(np.shape(X_train_all))
print(np.shape(y_train_all))
